# Test ratail

**Autor**: Edward Guevara \
**Fecha**: 06/15/2023 \
**Ultima modificacino**: 06/15/2023 \
**objetivos**: 
1. Describir los clientes de la tienda en base a su información transaccional
2. Crear un modelo que nos permita detectar a los clientes que son buenos compradores para ofrecerles descuentos personalizados.


# Introduccion

Cuando una tienda minorista (ratail), quiere retener clientes e incrementar las ventas, es necesario conocer el comportamiento de los clientes para ofrecerles ofertas personalizadas. En cuanto se consiga venderles **X** producto, venderles **Y** y **Z** productos sera mas facil. Por lo tanto, se requiere conocer cual son las variables que tienen mayor influencia para identificar buenos clientes y ofrecerles ofertas personalizadas.


Descripcion de las Variables:

1. transaction_data.csv
    * household_key: Identificación única del cliente.
    * BASKET_ID: Identificación única de la  transacción.
    * PRODUCT_ID: Identificación única del producto.
    * QUANTITY: Cantidad de productos comprados.
    * SALES_VALUE: Monto en dolares de la compra.
    * STORE_ID: Identificación única de cada tienda.
    * WEEK_NO: Semana en que se realizó la transacción (1 - 102)
2. product.csv
    * PRODUCT_ID: Identificación única del producto.
    * MANUFACTURER: Código del proveedor.
    * DEPARTMENT: Departamento al cual pertenece un producto .
    * BRAND: Indica si la marca del producto es internacional o nacional.
    * COMMODITY_DESC:Categoría al que pertenece el producto.
3. hh_demographic.csv
    * AGE_DESC: Rango de edad estimado.
    * MARITAL_STATUS_CODE: Estado marital del cliente.
    * INCOME_DESC: Ingresos del cliente.
    * household_key: ID por cada cliente
    * label: Indica si el cliente es buen comprador o no (0= mal comprador, 1=buen comprador)


#  Conocimientos basicos del dataset

## Setup

In [53]:
import pandas as pd
import numpy as np

## Importacion de datos

In [54]:
transaction_data = pd.read_csv("./dataset_retail_test_with_modelling/transaction.csv", sep = ',')
product_data = pd.read_csv("./dataset_retail_test_with_modelling/product.csv", sep = ',')
demographic_data = pd.read_csv("./dataset_retail_test_with_modelling/demographic.csv", sep = ',')

**transaction_data**

In [55]:
transaction_data.head()

,household_key,BASKET_ID,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,WEEK_NO
0,2375,26984851472,1004906,1,1.39,364,1
1,2375,26984851472,1033142,1,0.82,364,1
2,2375,26984851472,1036325,1,0.99,364,1
3,2375,26984851472,1082185,1,1.21,364,1
4,2375,26984851472,8160430,1,1.50,364,1


**product_data**

In [56]:
product_data.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC
0,25671,2,GROCERY,National,FRZN ICE
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION
2,26093,69,PASTRY,Private,BREAD
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE
4,26355,69,GROCERY,Private,COOKIES/CONES


**demographic_data**

In [57]:
demographic_data.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,household_key,label
0,65+,A,35-49K,1,0
1,45-54,A,50-74K,7,1
2,25-34,U,25-34K,8,0
3,25-34,U,75-99K,13,0
4,45-54,B,50-74K,16,0


Se uniran los tres dataset en las columnas correspondientes para obtener un solo dataset que contenga toda la informacion para el analisis.

In [58]:
df = pd.merge(transaction_data, product_data, on='PRODUCT_ID')
df = pd.merge(df, demographic_data, on='household_key')
df.head()

,household_key,BASKET_ID,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,WEEK_NO,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,label
0,1929,27021022215,1004906,1,1.39,441,1,69,PRODUCE,Private,POTATOES,35-44,B,25-34K,1
1,1929,27601005106,1004906,1,1.99,441,8,69,PRODUCE,Private,POTATOES,35-44,B,25-34K,1
2,1929,28110785584,1004906,1,1.00,292,14,69,PRODUCE,Private,POTATOES,35-44,B,25-34K,1
3,1929,30032381853,1004906,1,2.99,441,30,69,PRODUCE,Private,POTATOES,35-44,B,25-34K,1
4,1929,30035429101,1004906,1,2.99,292,30,69,PRODUCE,Private,POTATOES,35-44,B,25-34K,1


## Tipos de datos

In [59]:
print('Tipos de datos por columna: \n')
display(df.dtypes)

Tipos de datos por columna: 



household_key            int64
BASKET_ID                int64
PRODUCT_ID               int64
QUANTITY                 int64
SALES_VALUE            float64
STORE_ID                 int64
WEEK_NO                  int64
MANUFACTURER             int64
DEPARTMENT              object
BRAND                   object
COMMODITY_DESC          object
AGE_DESC                object
MARITAL_STATUS_CODE     object
INCOME_DESC             object
label                    int64
dtype: object

## Descripcion Estadistica

**Variables tipo numericas**

In [60]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
household_key,1427303.0,1.270911e+03,7.413937e+02,1.000000e+00,6.310000e+02,1.260000e+03,1.939000e+03,2.499000e+03
BASKET_ID,1427303.0,3.415043e+10,4.680249e+09,2.698490e+10,3.058042e+10,3.289548e+10,4.012860e+10,4.230271e+10
PRODUCT_ID,1427303.0,2.952598e+06,3.887443e+06,2.567100e+04,9.186635e+05,1.029743e+06,1.134533e+06,1.831630e+07
QUANTITY,1427303.0,1.148580e+02,1.252137e+03,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,8.963800e+04
SALES_VALUE,1427303.0,3.151199e+00,4.251720e+00,0.000000e+00,1.290000e+00,2.090000e+00,3.490000e+00,8.400000e+02
STORE_ID,1427303.0,3.352289e+03,9.243143e+03,1.000000e+00,3.330000e+02,3.750000e+02,4.240000e+02,3.428000e+04
WEEK_NO,1427303.0,5.709954e+01,2.668867e+01,1.000000e+00,3.500000e+01,5.800000e+01,8.000000e+01,1.020000e+02
MANUFACTURER,1427303.0,1.004945e+03,1.370763e+03,1.000000e+00,6.900000e+01,5.390000e+02,1.256000e+03,6.477000e+03
label,1427303.0,5.112790e-01,4.998729e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


**Variables tipo objeto**

In [61]:
df.describe(include=['object']).T

,count,unique,top,freq
DEPARTMENT,1427303,41,GROCERY,896407
BRAND,1427303,2,National,1025428
COMMODITY_DESC,1427303,305,SOFT DRINKS,60183
AGE_DESC,1427303,6,45-54,520586
MARITAL_STATUS_CODE,1427303,3,A,641796
INCOME_DESC,1427303,12,50-74K,348536


## Info

In [62]:
print('Informacion resumida del tipo de datos y valores no-null: \n')
display(df.info())

Informacion resumida del tipo de datos y valores no-null: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1427303 entries, 0 to 1427302
Data columns (total 15 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   household_key        1427303 non-null  int64  
 1   BASKET_ID            1427303 non-null  int64  
 2   PRODUCT_ID           1427303 non-null  int64  
 3   QUANTITY             1427303 non-null  int64  
 4   SALES_VALUE          1427303 non-null  float64
 5   STORE_ID             1427303 non-null  int64  
 6   WEEK_NO              1427303 non-null  int64  
 7   MANUFACTURER         1427303 non-null  int64  
 8   DEPARTMENT           1427303 non-null  object 
 9   BRAND                1427303 non-null  object 
 10  COMMODITY_DESC       1427303 non-null  object 
 11  AGE_DESC             1427303 non-null  object 
 12  MARITAL_STATUS_CODE  1427303 non-null  object 
 13  INCOME_DESC          1427303 non-null  obj

None

## Dataset final

Basado en los conocimientos basicos del datasets unificado. Este esta bastante limpio y listo para el analisis. Por lo tanto, se guardara el dataset unficado con el nombre `test_ratail.csv` 

In [63]:
df.to_csv('ratail_dataset.csv', index=False)